For this next tutorial we will dive  more into the parameter estimation performed by the fevd() function. 
As you might remember from the previous tutorials the GEV distribution has three variables: the location,
scale, and shape parameter. By default the fevd() function uses a 'maximum likelihood estimation' (MLE).
A MLE is an estimator, a function of your data that gives you an approximation of the parameters. In a 
MLE we compute the likelihood that a given set of parameters would return the observed data. the likelihood
is given by the earlier discussed probability density function that results from a set of parameters. 

As you might have noticed we thus made two very important assumptions in our previous tutorials... First
we assumed we were working with the GEV distribution, second we assumed the MLE method would give us the 
best parameter estimation. Here we will test these assumptions by first using different parameter estimation
methods to see if our parameter values signficiantly change. Next we show you how you can use different 
models.

But first lets open the precipitation data from Germany once more, and fit a GEV distribution to it using
the MLE method, you can do this by adding "method = MLE" to your fevd() function. Have a look at the parameter
values.

In [1]:
# !pip install -q condacolab
# import condacolab
# condacolab.install()
# #install dependencies - taken from <Yosmely Bermúdez> comments for Tutorial 6
# # We need this to install eigen which is needed for SDFC to install correctly
# !mamba install eigen numpy matplotlib seaborn pandas cartopy scipy texttable intake xarrayutils xmip cf_xarray intake-esm
# !pip install -v https://github.com/yrobink/SDFC/archive/master.zip#subdirectory=python
# !pip install https://github.com/njleach/mystatsfunctions/archive/master.zip


In [2]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
# import cartopy.crs as ccrs
from scipy import stats

In [3]:
import extremes_functions as ef
from mystatsfunctions import OLSE,LMoments
import SDFC as sd

In [4]:
def estimate_return_level(quantile,model):
    loc, scale, shape = model.coef_
    level = loc - scale / shape * (1 - (-np.log(quantile))**(-shape))
    # level = stats.genextreme.ppf(quantile,-shape,loc=loc,scale=scale)
    return level

In [5]:
import os, pooch

fname = 'precipitationGermany_1920-2022.csv'
if not os.path.exists(fname):
    url = "https://osf.io/xs7h6/download"
    fname = pooch.retrieve(url, known_hash=None)

data = pd.read_csv(fname, index_col=0).set_index('years')

data.columns=['precipitation']
precipitation = data.precipitation

In [6]:
fit, model = ef.fit_return_levels_sdfc(precipitation.values,times=np.arange(1.1,1000),periods_per_year=1,kind='GEV',N_boot=10,full=True,model=True)

There are two other parameter estimation methods that we will explore and compare with the MLE: the
L-moments method, and a Baseyian method. The L-moments describe the shape of a probability distribution,
just like regular moments, and are a linear combination of the order statistics (L stands for linear).
The GEV-distribution parameters can be computed through a set of equations based from these L-moments.

The Bayesian technique on the other hand, [explain]

Estimate the GEV parameters based on these two method by changing the 'method' to either "Lmoments" and 
"Bayesian". After that compute the 100-year flood for all three parameter sets (MLE, Lmoments and Bayesian)

In [7]:
fit_moments, model_moments = ef.fit_return_levels_sdfc(precipitation.values,times=np.arange(1.1,1000),periods_per_year=1,kind='GEV',N_boot=10,full=True,model=True,method='moments')

In [8]:
fit_moments

<xarray.Dataset>
Dimensions:            (return period: 999, N: 10, return_period_obs: 103)
Coordinates:
  * return period      (return period) float64 1.1 2.1 3.1 ... 997.1 998.1 999.1
  * N                  (N) int64 0 1 2 3 4 5 6 7 8 9
  * return_period_obs  (return_period_obs) float64 1.01 1.02 1.03 ... 52.0 104.0
Data variables:
    return level       (return period, N) float64 24.54 24.85 ... 40.41 40.98
    mu                 (N) float64 26.26 26.71 27.38 28.3 ... 26.73 26.06 27.43
    sigma              (N) float64 1.963 2.134 2.09 1.738 ... 2.131 2.077 1.962
    xi                 (N) float64 1e-08 1e-08 1e-08 1e-08 ... 1e-08 1e-08 1e-08
    return_level_obs   (return_period_obs) float64 15.6 16.2 16.9 ... 59.9 69.5

In [9]:
model_moments

+---------------+--------+------------+--------+----------------+----------------+
| GEV (moments) |  Link  |    Type    |  coef  | Quantile 0.025 | Quantile 0.975 |
+===============+========+============+========+================+================+
| loc           | IdLink | Stationary | 26.526 | 25.328         | 28.102         |
+---------------+--------+------------+--------+----------------+----------------+
| scale         | IdLink | Stationary | 2.042  | 1.788          | 2.134          |
+---------------+--------+------------+--------+----------------+----------------+
| shape         | IdLink | Stationary | 0      | 0              | 0              |
+---------------+--------+------------+--------+----------------+----------------+

In [10]:
prior = stats.multivariate_normal(mean= model.coef_, cov = np.cov(model.coefs_bootstrap.T), allow_singular=True)

In [11]:
fit_bayes, model_bayes = ef.fit_return_levels_sdfc(precipitation.values,times=np.arange(1.1,1000),periods_per_year=1,kind='GEV',N_boot=10,full=True,model=True,method='bayesian',prior=prior,mcmc_init=model.coef_)

/Users/wesleybanfield/miniconda3/envs/pangeo/lib/python3.10/site-packages/SDFC/__AbstractLaw.py:438: RuntimeWarning: invalid value encountered in double_scalars
  p_accept = np.exp( p_next - p_current )


In [12]:
fit_bayes

<xarray.Dataset>
Dimensions:            (return period: 999, N: 10, return_period_obs: 103)
Coordinates:
  * return period      (return period) float64 1.1 2.1 3.1 ... 997.1 998.1 999.1
  * N                  (N) int64 0 1 2 3 4 5 6 7 8 9
  * return_period_obs  (return_period_obs) float64 1.01 1.02 1.03 ... 52.0 104.0
Data variables:
    return level       (return period, N) float64 -3.555 1.416 ... 1.197 -7.195
    mu                 (N) float64 -3.741 0.6258 -3.48 ... 0.4064 1.043 -6.97
    sigma              (N) float64 -1.531 -1.044 -0.5835 ... 0.2801 -0.2929
    xi                 (N) float64 8.187 0.3402 -1.51 ... -2.098 -1.824 -1.299
    return_level_obs   (return_period_obs) float64 15.6 16.2 16.9 ... 59.9 69.5

In [13]:
model_bayes

+----------------+--------+------------+--------+----------------+----------------+
| GEV (bayesian) |  Link  |    Type    |  coef  | Quantile 0.025 | Quantile 0.975 |
+================+========+============+========+================+================+
| loc            | IdLink | Stationary | 26.268 | -6.244         | 16.056         |
+----------------+--------+------------+--------+----------------+----------------+
| scale          | IdLink | Stationary | 7.365  | -1.825         | 7.674          |
+----------------+--------+------------+--------+----------------+----------------+
| shape          | IdLink | Stationary | 0.052  | -4.118         | 7.266          |
+----------------+--------+------------+--------+----------------+----------------+

In [14]:
period = 100
quantile = 1-1/period

print('MLE: %.2f' % estimate_return_level(quantile,model))
print('Moments: %.2f' % estimate_return_level(quantile,model_moments))
print('Bayes: %.2f' % estimate_return_level(quantile,model_bayes))

MLE: 64.21
Moments: 35.92
Bayes: 64.51


In [15]:
period = 50
quantile = 1-1/period

print('MLE: %.2f' % estimate_return_level(quantile,model))
print('Moments: %.2f' % estimate_return_level(quantile,model_moments))
print('Bayes: %.2f' % estimate_return_level(quantile,model_bayes))

MLE: 57.92
Moments: 34.49
Bayes: 58.11


In [16]:
period = 500
quantile = 1-1/period

print('MLE: %.2f' % estimate_return_level(quantile,model))
print('Moments: %.2f' % estimate_return_level(quantile,model_moments))
print('Bayes: %.2f' % estimate_return_level(quantile,model_bayes))

MLE: 79.55
Moments: 39.21
Bayes: 80.23


### Fit Moments using a different method

In [17]:
gev = LMoments.gev()
gev.fit(precipitation.values)
gev.X, gev.a, gev.k

(26.453249703966794, 7.687907961826535, -0.01055113362649465)